In [1]:
using Gmsh: gmsh
using Gridap
using GridapGmsh

In [2]:
const L = 25.5      #Length
const H = 19.1      #Height
const Lu = 4.6      #offset of load from left end
const Lh = 11.5     #notch length
const lsp = 0.08    #length factor
const CP = H/2      #center height  
const CH = 0.46     #Crack height
const hf = 0.06/2.1    #finer mesh size
const hfc = 0.06/50    #finer mesh size
const h = 60*hf     #coarser mesh size
const Lw = 2*h         #load width
const FMR = CH/2 

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
p1 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h)  
p2 = gmsh.model.geo.addPoint(Lu-0.5*Lw, 0.0, 0.0, h)
p3 = gmsh.model.geo.addPoint(Lu+0.5*Lw, 0.0, 0.0, h) 
p4 = gmsh.model.geo.addPoint(L, 0, 0.0, h) 
p5 = gmsh.model.geo.addPoint(L, H, 0.0, h)
p6 = gmsh.model.geo.addPoint(Lu+0.5*Lw, H, 0.0, h)
p7 = gmsh.model.geo.addPoint(Lu-0.5*Lw, H, 0.0, h)
p8 = gmsh.model.geo.addPoint(0, H, 0.0, h)
p9 = gmsh.model.geo.addPoint(0, CP + (CH*0.5), 0.0, h)
p10 = gmsh.model.geo.addPoint(Lh-(CH*0.5), CP + (CH*0.5), 0.0, h)
p11 = gmsh.model.geo.addPoint(Lh, CP, 0.0, hfc)
p12 = gmsh.model.geo.addPoint(Lh-(CH*0.5), CP - (CH*0.5), 0.0, h)
p13 = gmsh.model.geo.addPoint(0, CP - (CH*0.5), 0.0, h)

pp14 = gmsh.model.geo.addPoint(L, CP, 0.0, h)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p5)
l5 = gmsh.model.geo.addLine(p5, p6)
l6 = gmsh.model.geo.addLine(p6, p7)
l7 = gmsh.model.geo.addLine(p7, p8)
l8 = gmsh.model.geo.addLine(p8, p9)
l9 = gmsh.model.geo.addLine(p9, p10)
l10 = gmsh.model.geo.addLine(p10, p11)
l11 = gmsh.model.geo.addLine(p11, p12)
l12 = gmsh.model.geo.addLine(p12, p13)
l13 = gmsh.model.geo.addLine(p13, p1)

ll1 = gmsh.model.geo.addLine(p11, pp14)

cl1 = gmsh.model.geo.addCurveLoop([l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,l13])

ps1 = gmsh.model.geo.addPlaneSurface([cl1])

pg1 = gmsh.model.addPhysicalGroup(2, [ps1])
pg2 = gmsh.model.addPhysicalGroup(1, [l2])
pg3 = gmsh.model.addPhysicalGroup(1, [l6])
pg4 = gmsh.model.addPhysicalGroup(1, [l7,l6,l5])
pg5 = gmsh.model.addPhysicalGroup(1, [l1,l2,l3])

gmsh.model.setPhysicalName(2, pg1, "Domain")
gmsh.model.setPhysicalName(1, pg2, "DirichletBot")
gmsh.model.setPhysicalName(1, pg3, "DirichletTop")
gmsh.model.setPhysicalName(1, pg4, "ElectricPotentialTop")
gmsh.model.setPhysicalName(1, pg5, "ElectricPotentialBot")

gmsh.model.mesh.field.add("Box", 11)
gmsh.model.mesh.field.setNumber(11, "VIn", hf)
gmsh.model.mesh.field.setNumber(11, "VOut", h)
gmsh.model.mesh.field.setNumber(11, "XMin", 0.95*Lh)
gmsh.model.mesh.field.setNumber(11, "XMax", L)
gmsh.model.mesh.field.setNumber(11, "YMin", CP-4*FMR)
gmsh.model.mesh.field.setNumber(11, "YMax", CP+4*FMR)

gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "EdgesList", [ll1])

gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", hf)
gmsh.model.mesh.field.setNumber(2, "LcMax", h)
gmsh.model.mesh.field.setNumber(2, "DistMin", 4*FMR)
gmsh.model.mesh.field.setNumber(2, "DistMax", 1)

gmsh.model.mesh.field.setAsBackgroundMesh(2)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("PlateWithEdgeNotch.msh")
gmsh.finalize()

Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Meshing curve 2 (Line)
Info    : Meshing curve 3 (Line)
Info    : Meshing curve 4 (Line)
Info    : Meshing curve 5 (Line)
Info    : Meshing curve 6 (Line)
Info    : Meshing curve 7 (Line)
Info    : Meshing curve 8 (Line)
Info    : Meshing curve 9 (Line)
Info    : Meshing curve 10 (Line)
Info    : Meshing curve 11 (Line)
Info    : Meshing curve 12 (Line)
Info    : Meshing curve 13 (Line)
Info    : Meshing curve 14 (Line)
Info    : Done meshing 1D (0.009476 s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Delaunay)
Info    : Done meshing 2D (1.01656 s)
Info    : 34436 vertices 68375 elements
Info    : Writing 'PlateWithEdgeNotch.msh'...
Info    : Done writing 'PlateWithEdgeNotch.msh'


In [3]:
model = GmshDiscreteModel("PlateWithEdgeNotch.msh")
writevtk(model,"PlateWithEdgeNotch")

Info    : Reading 'PlateWithEdgeNotch.msh'...
Info    : 29 entities
Info    : 33927 nodes
Info    : 67748 elements
Info    : Done reading 'PlateWithEdgeNotch.msh'


3-element Array{Array{String,1},1}:
 ["PlateWithEdgeNotch_0.vtu"]
 ["PlateWithEdgeNotch_1.vtu"]
 ["PlateWithEdgeNotch_2.vtu"]